In [81]:
#pip install mysql-connector-python

In [33]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [34]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [35]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [36]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [37]:
c = create_db_connection("giniewicz.it","team06","te@m0g","team06")

MySQL Database connection successful


In [38]:
#suma zarobionych pieniędzy ze sprzedaży gier w styczniu 2022
q1 = """
SELECT sum(game_purchase.price)
FROM game_purchase
WHERE DATE(game_purchase.purchase_date) BETWEEN '2022-01-01' AND '2022-01-31'
"""

#suma zarobionych pieniędzy z wypożyczeń gier w styczniu 2022
q2 = """
SELECT sum(game_rental.price)
FROM game_rental
WHERE DATE(game_rental.rental_date) BETWEEN '2022-01-01' AND '2022-01-31'
"""

# suma zarobków pracowników
q3 = """
SELECT sum(salary)
FROM employees
"""

#pracownicy
q4 = """
SELECT *
FROM employees
"""

#za ile pieniędzy w danym miesiącu pracownik sprzedał gier
q5 = """
SELECT sum(price) as t, employee_id, month(purchase_date) as m, year(purchase_date) as y, 
concat(employee_id,month(purchase_date),year(purchase_date)) as id
FROM game_purchase
GROUP BY y, m, employee_id
"""

#za ile pieniędzy w danym miesiącu pracownik wypożyczył gier
q6 = """
SELECT sum(game_rental.price) as t, game_rental.employee_id, month(game_rental.rental_date) as m, year(game_rental.rental_date) as y, 
concat(game_rental.employee_id,month(game_rental.rental_date),year(game_rental.rental_date)) as id,
concat(employees.first_name," ",employees.last_name) as names
FROM game_rental INNER JOIN employees on game_rental.employee_id = employees.employee_id
GROUP BY y, m, game_rental.employee_id
"""

#przychody ze sprzedaży
q7 = """
SELECT sum(game_purchase.price), shop_inventory.game_title
FROM game_purchase INNER JOIN shop_inventory ON game_purchase.game_id = shop_inventory.game_id
GROUP BY game_purchase.game_id
ORDER BY sum(game_purchase.price) DESC
LIMIT 5;
"""

#przychody z wypożyczeń
q8 = """
SELECT sum(game_rental.price), shop_inventory.game_title
FROM game_rental INNER JOIN shop_inventory ON game_rental.game_id = shop_inventory.game_id
GROUP BY game_rental.game_id
ORDER BY sum(game_rental.price) DESC
LIMIT 5;
"""

#top 10 zawodników szachów (inna gra - inne game_id)
q9 = """
SELECT t.total_score, k.first_name, k.last_name
FROM (
  SELECT SUM(score) AS total_score, customer_id
  FROM tournaments__scores
  INNER JOIN game_tournaments ON tournaments__scores.tournament_id = game_tournaments.tournament_id
  WHERE game_id = 1
  GROUP BY customer_id
  ORDER BY total_score DESC
  LIMIT 10
) AS t
INNER JOIN customers AS k ON t.customer_id = k.customer_id;
"""
# 5 klientów, którzy wydali w naszych sklepach najwięcej i ich wydatek podzielony przez ilosc dni miedzy pierwszym a ostatnim zakupem
q10 = """
SELECT c.first_name, c.last_name, ROUND(SUM(gp.price) / DATEDIFF(MAX(gp.purchase_date), MIN(gp.purchase_date)), 2) AS expenditure_per_day
FROM customers c
JOIN game_purchase gp ON c.customer_id = gp.customer_id
GROUP BY c.customer_id
ORDER BY SUM(gp.price) DESC
LIMIT 5;

"""

q11 = """

SELECT c.first_name, c.last_name, si.game_title, DATEDIFF(MAX(gr.return_date), MIN(gr.rental_date)) AS days_rented
FROM customers c
JOIN game_rental gr ON c.customer_id = gr.customer_id
JOIN shop_inventory si ON gr.game_id = si.game_id
GROUP BY c.customer_id, si.game_id
ORDER BY days_rented DESC
LIMIT 10;
"""

q12 = """
SELECT rental_date, COUNT(rental_date) AS count
FROM game_rental
GROUP BY rental_date
ORDER BY count DESC
LIMIT 3;

"""


In [39]:
r1 = float(read_query(c,q1)[0][0])
r2 = float(read_query(c,q2)[0][0])
r3 = float(read_query(c,q3)[0][0])
r4 = read_query(c,q4)
r5 = read_query(c,q5)
r6 = read_query(c,q6)
r7 = read_query(c,q7)
r8 = read_query(c,q8)
r9 = read_query(c,q9)
r10 = read_query(c, q10)
r11 = read_query(c, q11)
r12 = read_query(c, q12)

In [43]:
df1 = pd.DataFrame({
    "przychód ze sprzedaży": [r1],
    "przychód z wypożyczeń": [r2],
    "koszt utrzymania pracowników": [r3]})
df4 = pd.DataFrame(r4, columns =['ID pracownika', "imię", "nazwisko", 'wypłata'])
df5 = pd.DataFrame(r5, columns =['cena kupna gry', 'ID pracownika', 'miesiąc', 'rok',"ID"])
df6 = pd.DataFrame(r6, columns =['cena wypożyczenia gry', 'ID pracownika', 'miesiąc', 'rok',"ID","imię"])
#df6_1 = df6
df6_1 = pd.merge(df5, df6, on='ID')
#df6_1 = df6.join(df5.set_index('ID pracownika'), on='ID')
df7 = pd.DataFrame(r7, columns =["przychód ze sprzedaży","nazwa gry"])
df8 = pd.DataFrame(r8, columns =["przychód z wypożyczeń","nazwa gry"])
df9 = pd.DataFrame(r9, columns =["całkowita punktacja", "imię", "nazwisko"])
df10 = pd.DataFrame(r10, columns = ["imię", "nazwisko", " znormalizowany wydatek"])
df11 = pd.DataFrame(r11, columns = ["imię", "nazwisko","tytuł gry", " dni wypożyczenia"])
df12 = pd.DataFrame(r12, columns = ["data", "ilość wypożyczeń"])

In [44]:
# zysk za styczeń 2022
s = r1+r2-r3

In [45]:
df6_1

,cena kupna gry,ID pracownika_x,miesiąc_x,rok_x,ID,cena wypożyczenia gry,ID pracownika_y,miesiąc_y,rok_y,imię
0,285.00,1,4,2020,142020,450.00,1,4,2020,Adam Plotnik
1,150.00,2,4,2020,242020,735.00,2,4,2020,Błażej Słowik
2,520.00,3,4,2020,342020,695.00,3,4,2020,Wojtek Siłka
3,1200.00,1,5,2020,152020,1750.00,1,5,2020,Adam Plotnik
4,1470.00,2,5,2020,252020,1690.00,2,5,2020,Błażej Słowik
...,...,...,...,...,...,...,...,...,...,...
102,3050.00,4,2,2022,422022,3515.00,4,2,2022,Wiktoria Wilk
103,3510.00,5,2,2022,522022,3040.00,5,2,2022,Oskar Rogowski
104,3330.00,6,2,2022,622022,3100.00,6,2,2022,Marcin Nowak
105,4135.00,7,2,2022,722022,3025.00,7,2,2022,Paweł Chałupka


In [51]:
# top pracownicy
df6_1["cena wypożyczenia gry"] = df6_1["cena wypożyczenia gry"].fillna(0)
df6_1["pełny przychód"] =( df6_1["cena wypożyczenia gry"] + df6_1["cena kupna gry"]).astype(int)
max_value_idx = df6_1.groupby(['rok_x', 'miesiąc_x'])['pełny przychód'].idxmax()
max_value_employees = df6_1.loc[max_value_idx][["imię","pełny przychód","miesiąc_x","rok_x"]].rename(columns={"name": "top_employee"})

In [80]:
html = f'''
<html>
    <head>
        <title>Wypożyczalnia gier planszowych</title>
        <style>
         table {{
      border-collapse: collapse;
    }}

    td {{
      text-align: center;
      vertical-align: middle;
    }}
    
    .col {{
  display: inline-block;
}}
        
.table-container {{
            width: 50%;
            margin: 0 auto;
        }}
        .table {{
  width: 100%;
  border-collapse: collapse;
}}

.table th,
.table td {{
  padding: 10px;
  border: 1px solid #ccc;
}}

.table th {{
  background-color: #f2f2f2;
  color: #333;
  font-weight: bold;
}}

.table tr:nth-child(even) {{
  background-color: #f9f9f9;
}}

.table tr:first-child {{
  font-weight: bold;
}}


            .highlight {{
                font-weight: bold;
                color: green;
            }}
        </style>
    </head>
    <body>
        <h1>Raport - analiza danych.</h1>
        Baza danych, która powstała zawiera 7 tabel (customers, employees, game_purchase,
        game_rental, game_tournaments, shop_inventory, tournaments_scores).
        Poniżej przedstawiamy analizę danych w niej zawartych.
        <h2>1. Pracownicy miesiąca</h2>
        {max_value_employees.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center" )}
        
        <h2>2. Top 10 zawodników</h2>
        <p>Najlepsi zawodnicy w turnieju szachowym:</p>
        {df9.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
        
        <h2>3. Przychody w zależności od sprzedaży i wypożyczeń</h2>
        <p>Największe przychody ze sprzedaży:</p>
        {df7.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
        <p>Największe przychody z wypożyczeń:</p>
        {df8.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
        
        <h2>4. Własne</h2>
        <h3>4.1 Zyski za styczeń 2022</h3>
        {df1.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
        <p>Zyski z działalności wypożyczalni gier w styczniu 2022,
        obliczone jako różnica między sumą z przychodów ze sprzedaży i wypożyczeń gier,
        a sumą miesięcznych pensji pracowników:</p>
        <p><span class="highlight">{s} zł</span></p>
        
        <h3>4.2 Pięciu klientów, którzy wydali najwięcej w naszych sklepach oraz ich wydatek
        podzielony przez ilość dni między pierwszym a ostatnim zakupem</h3>
        <p> wyniki zostały uporządkowane malejąco a także zaokrąglone do dwóch miejsc po przecinku </p>
        {df10.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
        
        <h3> 4.3 Gry z najdłuższym okresem wypożyczenia </h3>
        {df11.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
        
            <h3> 4.4 Trzy dni, w których najczęściej wypożyczano gry</h3>
        {df12.to_html(classes = 'table table-stripped col', col_space = "auto", index = False, justify = "center")}
    </body>
</html>
    '''

with open('html_report.html', 'w') as f:
    f.write(html)